# 机器学习慕课吴恩达-实验3-多类分类

### 1.从.mat文件中加载数据

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat


In [2]:
data = loadmat('ex3data1.mat')
data


{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [3]:
data['X'].shape, data['y'].shape

((5000, 400), (5000, 1))

X has size 5000 x 400  
y has size 5000 x 1

### 2.定义损失函数

In [4]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))


代价函数：
$J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{[-{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]}$

In [5]:
def cost(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    m = np.size(y)              # m=5000

    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / (2 * m)) * np.sum(np.power(theta[:, 1:theta.shape[1]], 2))
    return np.sum(first - second) / m + reg


梯度下降函数：  
\begin{align}
  & Repeat\text{ }until\text{ }convergence\text{ }\!\!\{\!\!\text{ } \\ 
 & \text{     }{{\theta }_{0}}:={{\theta }_{0}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{_{0}}^{(i)}} \\ 
 & \text{     }{{\theta }_{j}}:={{\theta }_{j}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{j}^{(i)}}+\frac{\lambda }{m}{{\theta }_{j}} \\ 
 & \text{          }\!\!\}\!\!\text{ } \\ 
 & Repeat \\ 
\end{align}

In [6]:
# def gradient(theta, X, y, lambdaP):
#     theta = np.matrix(theta)    # 1*401 or 10*401?
#     X = np.matrix(X)            # 5000*401
#     y = np.matrix(y)            # 5000*1

#     theta1 = theta
#     theta1[:, 0] = 0

#     m = np.size(y)              # m=5000
#     # h = sigmoid(X * theta.T)
#     h = sigmoid(np.matmul(X, theta.T))  # 5000*1

#     # 在这，grad形状为401*1
#     # (401*5000 .* 5000*1 )T--> 1*401
#     # grad = (1 / m) * np.sum(X.T * (h - y)).T
#     grad = ((X.T * (h - y)) / len(X)).T
#     grad = grad + lambdaP / m * theta1
#     # grad[0, 0] = np.sum(np.multiply(h-y, X[:, 0])) / len(X)

#     return grad
   
def gradient(theta, X, y, learningRate):
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y

    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)

    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:, 0])) / len(X)

    return np.array(grad).ravel()

In [7]:
# testTheta = np.matrix([[1,2,3],[4,5,6]])
# testTheta[: ,0] = 0
# print(testTheta)


### 3.构建分类器

In [8]:
from scipy.optimize import minimize


def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]
    params = X.shape[1]

    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))

    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(
            X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1, :] = fmin.x

    return all_theta


### 4.计算theta并计算预测准确率

In [9]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-2.38090200e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30362219e-03, -6.06149084e-10,  0.00000000e+00],
       [-3.18420805e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.46211711e-03, -5.08723221e-04,  0.00000000e+00],
       [-4.79563739e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.86475853e-05, -2.47347362e-07,  0.00000000e+00],
       ...,
       [-7.98603898e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.94272873e-05,  7.21573274e-06,  0.00000000e+00],
       [-4.57361264e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33685653e-03,  1.00046175e-04,  0.00000000e+00],
       [-5.40406502e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16595999e-04,  7.89322344e-06,  0.00000000e+00]])

预测：

In [10]:
def predict_all(X, all_theta):
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]

    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)

    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)

    # create array of the index with the maximum probability
    # 这个argmax是返回对应轴的最大元素索引
    h_argmax = np.argmax(h, axis=1)

    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1

    return h_argmax

In [11]:
y_pred = predict_all(data['X'], all_theta)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))

accuracy = 94.46%
